Git clone tensorflows models

In [0]:
git clone https://github.com/tensorflow/models.git

Move to your research directory

In [0]:
cd models/research/

In [0]:
protoc object_detection/protos/*.proto --python_out=.

In [0]:
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

In [0]:
cd slim

In [0]:
python3 setup.py build

In [0]:
python3 setup.py install

In [0]:
cd ..

Create a image folder inside object_detection folder. Inside your image folder create two another folder for training images and testing images. 
Copy your all images in training folder.
Go to [link](https://tzutalin.github.io/labelImg/) and download labelimg. After donwload, annotate your all images in xml format from training folder . After annotate your images, select random 20% of images and their corresponding xml file from training folder and cut them to your test folder inside your images folder

Copy xml2csv.py from 3rd-eye/records_and_csv/ folder and run it from your object_detection folder. After this process, you should have two csv files inside your images folder

Download csv to tf-records convertor from [tfrecord](https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py) or you can use  	tf_record.py from 3rd-eye/records_and_csv/ folder. Run it as given syntax below

In [0]:
python3 tfrec.py --csv_input=<path_to_your_train_csv_file> --image_dir=<path_to_your_train_images> --output_path=<output_path_to_your_train_records_file>

In [0]:
python3 tfrec.py --csv_input=<path_to_your_test_csv_file> --image_dir=<path_to_your_test_images> --output_path=<output_path_to_your_test_records_file>

After this process, you should have two .records file

Go to [models]((https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) and choose your model. In my case i used to faster_rcnn_nas

In [0]:
wget http://download.tensorflow.org/models/object_detection/faster_rcnn_nas_coco_2018_01_28.tar.gz

In [0]:
tar -xvf faster_rcnn_nas_coco_2018_01_28.tar.gz #your model name

Create a xyz folder inside your object_detection folder. Inside your xyz folder, create label.pbtxt file

```
item {
  id: 1
  name: 'your 1st class name '
}

item {
  id: 2
  name: 'your 2nd class name'
}

item {
  id: 3
  name: 'your 3rd class name'
}

.. and so on

```

Go to [configs](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) or go to your models/research/object_detection/samples/config/ and open your model's corresponding config file.In my case i used to faster_rcnn_nas_coco.config and save your config file in xyz folder after the following instruction 

Change following lines in your config file

1:
```
 num_classes: number of classes of your objects
```
2:
```
 fine_tune_checkpoint:"PATH_TO_BE_CONFIGURED/model.ckpt". In my case i had configured "faster_rcnn_nas_coco_2018_01_28/model.ckpt"
```

3:
```
train_input_reader: {
  tf_record_input_reader {
    input_path: "path to your train.records"
  }
  label_map_path: "path to your lables.pbtxt"
}
```
4:
```
eval_input_reader: {
  tf_record_input_reader {
    input_path: "path to your test.records"
  }
  label_map_path: "path to your labels.pbtxt"
  shuffle: false
  num_readers: 1
}

```

Copy train.py from object_detection/legacy directory and paste it into your research directory. Run train.py from your research directory as given syntax below

In [0]:
python3 train.py --logtostderr --train_dir=object_detection/xyz/ --pipeline_config_path=<path_to_your_config_file>

After you satisfy with your average loss (recommend avg-loss < 0.06) you can press 'ctrl+M' or stop your training

In [0]:
python3 export_inference_graph.py --input_type image_tensor --pipeline_config_path <path_to_your_config_file(.config)> --trained_checkpoint_prefix <path_to_your_trained_model_higher_checkpoint (.ckpt)> --output_directory <your output model name>

After the above process you should have  (your_model_name)folder 